In [23]:
import scipy as sp
import statistics
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import t
from scipy.stats import norm
import os
path = os.getcwd()



## Problem 1:

In [ ]:

# library.py

In [8]:
import library as lib

## Problem 2:

#### a. Using a normal distribution with an exponentially weighted variance (lambda=0.97); 

In [37]:
df = pd.read_csv('/Users/ruoqiyan/Documents/545_Assignment_Repository/Week05/problem1.csv')
# Using a normal distribution with an exponentially weighted variance (lambda=0.97)
λ = 0.97
weights_ = lib.populateWeights(df,λ)

In [43]:
#x = weights_ * df['x']
x = df['x'] * weights_

x = lib.simulate_normal(x,)
VaR_weighted = lib.VaR_distribution(x)
print('VaR_weighted: ', VaR_weighted)
ES_weighted = lib.ES_distribution(x)
print('ES_weighted: ', ES_weighted)

TypeError: simulate_normal() missing 1 required positional argument: 'cov'

#### b. Using a MLE fitted T distribution

In [30]:
x = df['x']
model = lib.fit_general_t(x)

print("VaR_MLEt", lib.VaR_distribution(model.error_model)) 
print("ES_MLEt", lib.ES_distribution(model.error_model))

VaR_MLEt 0.07647602684516216
ES_MLEt 0.11321790139118341


#### c. Using a Historic Simulation 

In [49]:
x = df['x']

def calculate_var_historical(returns, confidence_level, method):
    # Sort returns in ascending order
    returns_sorted = np.sort(returns)

    # Determine VaR observation index
    index = int(len(returns_sorted) * (1 - confidence_level))

    # Calculate VaR
    if method == 'var':
        var = -returns_sorted[index]
    elif method == 'es':
        var = lib.ES(returns_sorted)
    return var

confidence_level = 0.95
# Calculate VaR using historical simulation
print("VaR_his", calculate_var_historical(x, confidence_level, 'var')) 
print("ES_his", calculate_var_historical(x, confidence_level, 'es'))


VaR_his 0.075861511162783
ES_his 0.11677669788562187


#### Difference

## Problem 3:

In [50]:
df1 = pd.read_csv('/Users/ruoqiyan/Documents/545_Assignment_Repository/Week05/portfolio.csv')
df2 = pd.read_csv('/Users/ruoqiyan/Documents/545_Assignment_Repository/Week05/DailyPrices.csv')

In [58]:
import pandas as pd
import numpy as np

def return_calculate(prices, method="DISCRETE", dateColumn="date"):
    # Check if dateColumn is in the DataFrame
    if dateColumn not in prices.columns:
        raise ValueError(f"dateColumn: {dateColumn} not in DataFrame: {prices.columns}")
    
    # Exclude the date column from the calculation
    vars = [col for col in prices.columns if col != dateColumn]

    # Convert prices to a matrix (numpy array) for calculations
    p = prices[vars].values
    n, m = p.shape

    # Initialize an empty array for the calculated returns
    p2 = np.empty((n-1, m))

    # Calculate the price ratios
    for i in range(n-1):
        for j in range(m):
            p2[i, j] = p[i+1, j] / p[i, j]

    # Apply the specified return calculation method
    if method.upper() == "DISCRETE":
        p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError(f"method: {method} must be in (\"LOG\", \"DISCRETE\")")

    # Extract the dates for the output DataFrame
    dates = prices[dateColumn].iloc[1:].values

    # Create the output DataFrame
    out = pd.DataFrame(data=p2, columns=vars)
    out.insert(0, dateColumn, dates)
    
    return out


In [227]:
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

# Load the CSV file into a DataFrame
prices = pd.read_csv('/Users/ruoqiyan/Documents/545_Assignment_Repository/Week05/DailyPrices.csv')

# Current Prices
current_prices = prices.iloc[-1]

# Discrete returns
returns = return_calculate(prices, dateColumn="Date")

#### Portfolio A

In [228]:
df_A = df1[df1['Portfolio']=='A']
A_stock = list(df_A['Stock'].unique())

In [229]:
stocks = A_stock
# Get names of the columns except 'Date' and 'PLD'
nms = [nm for nm in stocks if nm not in ["Date", "PLD"]]
#nms.append('SPY')
returns = returns[nms]

# Setup how much we hold
Portfolio = pd.DataFrame({'stock': stocks, 'holding': np.ones(len(stocks))})

# Remove the mean from all returns
for nm in nms:
    v = returns[nm]
    returns[nm] = v - v.mean()


In [230]:
# Fit model for all stocks
fittedModels = {}

#fittedModels["SPY"] = lib.fit_general_t(returns['SPY'])

for stock in nms:
    fittedModels[stock] = lib.fit_general_t(returns[stock])


In [231]:
from scipy.stats import spearmanr, norm

# nms is a list of model names
U = pd.DataFrame()
for nm in nms:
    U[nm] = fittedModels[nm].u

# Calculate Spearman's rank correlation matrix
R = spearmanr(U.values).correlation

# Check the rank of R
evals = np.linalg.eigvals(R)
if min(evals) > -1e-8:
    print("Matrix is PSD")
else:
    print("Matrix is not PSD")

Matrix is PSD


In [232]:
# Simulation
NSim = 5000

simU = pd.DataFrame(
    norm.cdf(lib.simulate_pca(R, NSim)), 
    columns=nms
)


#simulatedReturns = pd.DataFrame({'SPY': fittedModels["SPY"].eval(simU['SPY'])})
simulatedReturns = pd.DataFrame()

for stock in nms:
    simulatedReturns[stock] = fittedModels[stock].eval(simU[stock])


In [233]:
iteration = [i for i in range(1, NSim + 1)]  # Adjusted for zero-based indexing
values = pd.merge(Portfolio.assign(key=0), pd.DataFrame({'iteration': iteration}).assign(key=0), on='key').drop('key', axis=1)

nVals = len(values)
currentValue = np.empty(nVals)
simulatedValue = np.empty(nVals)
pnl = np.empty(nVals)

for i in range(nVals):  # Adjusted for zero-based indexing
    price = current_prices[values.loc[i, 'stock']]
    holding = values.loc[i, 'holding']
    currentValue[i] = holding * price
    iteration = values.loc[i, 'iteration']-1
    simulatedValue[i] = holding * price * (1.0 + simulatedReturns.loc[iteration, values.loc[i, 'stock']])
    pnl[i] = simulatedValue[i] - currentValue[i]

values['currentValue'] = currentValue
values['simulatedValue'] = simulatedValue
values['pnl'] = pnl


In [234]:
values

,stock,holding,iteration,currentValue,simulatedValue,pnl
0,AAPL,1.0,1,150.639999,153.970027,3.330027
1,AAPL,1.0,2,150.639999,154.818403,4.178403
2,AAPL,1.0,3,150.639999,151.249100,0.609101
3,AAPL,1.0,4,150.639999,147.929285,-2.710714
4,AAPL,1.0,5,150.639999,145.271388,-5.368611
...,...,...,...,...,...,...
174995,TJX,1.0,4996,80.760002,79.390958,-1.369044
174996,TJX,1.0,4997,80.760002,78.714498,-2.045504
174997,TJX,1.0,4998,80.760002,80.170617,-0.589386
174998,TJX,1.0,4999,80.760002,80.524515,-0.235487


In [235]:
gdf = values.groupby('iteration')
# aggregate to totals per simulation iteration
totalValues = gdf.agg(
    currentValue=('currentValue', 'sum'),
    simulatedValue=('simulatedValue', 'sum'),
    pnl=('pnl', 'sum')
).reset_index()

In [236]:
x = totalValues['pnl']
current = totalValues['currentValue'].iloc[0]
print('Portfolio A')
ttl_var = lib.VaR(x) #153.9002522009032
print('total VaR:', ttl_var)
#lib.var.append(ttl_var)
ttl_es = lib.ES(x) # 202.22401640388208
print('total ES:', ttl_es)
#es.append(ttl_es)
ttl_var_pct = lib.VaR(x)/current #0.030780050440180638
print('total VaR_pct:', ttl_var_pct)
#var_pct.append(ttl_var_pct)
ttl_es_pct =lib.ES(x)/current #0.04044480328077642
print('total ES_pct:', ttl_es_pct)
#es_pct.append(ttl_es_pct)

Portfolio A
total VaR: 196.90687923751335
total ES: 256.9763649278964
total VaR_pct: 0.02998014305176433
total ES_pct: 0.03912604887799663


In [239]:
def agg(ptf, distribution):
    returns = return_calculate(prices, dateColumn="Date")

    df_A = df1[df1['Portfolio']==ptf]
    A_stock = list(df_A['Stock'].unique())

    stocks = A_stock
    # Get names of the columns except 'Date' and 'PLD'
    nms = [nm for nm in stocks if nm not in ["Date", "PLD"]]
    nms.append('SPY')
    returns = returns[nms]

    # Setup how much we hold
    Portfolio = pd.DataFrame({'stock': stocks, 'holding': np.ones(len(stocks))})

    # Remove the mean from all returns
    for nm in nms:
        v = returns[nm]
        returns[nm] = v - v.mean()

    # Fit model for all stocks
    fittedModels = {}
    #fittedModels["SPY"] = lib.fit_normal(returns['SPY'])

    if distribution == 'fit_general_t':
        for stock in nms:
            fittedModels[stock] = lib.fit_general_t(returns[stock])

    else:
        for stock in nms:
            fittedModels[stock] = lib.fit_normal(returns[stock])



    from scipy.stats import spearmanr, norm

    # nms is a list of model names
    U = pd.DataFrame()
    for nm in nms:
        U[nm] = fittedModels[nm].u

    # Calculate Spearman's rank correlation matrix
    R = spearmanr(U.values).correlation

    # Check the rank of R
    evals = np.linalg.eigvals(R)
    if min(evals) > -1e-8:
        print("Matrix is PSD")
    else:
        print("Matrix is not PSD")

    # Simulation
    NSim = 5000

    simU = pd.DataFrame(
        norm.cdf(lib.simulate_pca(R, NSim)),  # Convert standard normals to U
        columns=nms
    )
    simulatedReturns = pd.DataFrame()
    for stock in nms:
        # Assuming eval for other stocks takes two parameters: SPY's simulated returns and the stock's own simulations
        simulatedReturns[stock] = fittedModels[stock].eval(simU[stock])


    iteration = [i for i in range(1, NSim + 1)]  # Adjusted for zero-based indexing
    values = pd.merge(Portfolio.assign(key=0), pd.DataFrame({'iteration': iteration}).assign(key=0), on='key').drop('key', axis=1)

    nVals = len(values)
    currentValue = np.empty(nVals)
    simulatedValue = np.empty(nVals)
    pnl = np.empty(nVals)

    for i in range(nVals):  # Adjusted for zero-based indexing
        price = current_prices[values.loc[i, 'stock']]
        holding = values.loc[i, 'holding']
        currentValue[i] = holding * price
        iteration = values.loc[i, 'iteration']-1
        simulatedValue[i] = holding * price * (1.0 + simulatedReturns.loc[iteration, values.loc[i, 'stock']])
        pnl[i] = simulatedValue[i] - currentValue[i]

    values['currentValue'] = currentValue
    values['simulatedValue'] = simulatedValue
    values['pnl'] = pnl

    gdf = values.groupby('iteration')
    # aggregate to totals per simulation iteration
    totalValues = gdf.agg(
        currentValue=('currentValue', 'sum'),
        simulatedValue=('simulatedValue', 'sum'),
        pnl=('pnl', 'sum')
    ).reset_index()

    x = totalValues['pnl']
    current = totalValues['currentValue'].iloc[0]
    print('Portfolio'+ ptf)
    ttl_var = lib.VaR(x) #153.9002522009032
    print('total VaR:', ttl_var)
    #lib.var.append(ttl_var)
    ttl_es = lib.ES(x) # 202.22401640388208
    print('total ES:', ttl_es)
    #es.append(ttl_es)
    ttl_var_pct = lib.VaR(x)/current #0.030780050440180638
    print('total VaR_pct:', ttl_var_pct)
    #var_pct.append(ttl_var_pct)
    ttl_es_pct =lib.ES(x)/current #0.04044480328077642
    print('total ES_pct:', ttl_es_pct)
    #es_pct.append(ttl_es_pct)


   


In [240]:
agg('A','fit_general_t')

Matrix is PSD
PortfolioA
total VaR: 200.13018663416509
total ES: 262.18885925990065
total VaR_pct: 0.030470909129748154
total ES_pct: 0.03991967948315949


In [241]:
agg('C','fit_normal')

Matrix is PSD


KeyError: 'PLD'